In [67]:
from HelpFunctions.date_and_time import most_recent_thursday, split_time
from DAX.HelpFunctions.get_dax_data import get_dax_data
from datetime import timedelta
from HelpFunctions.calc_score import evaluate_horizon
import pandas as pd
from DAX.Models.baseline import baseline
import pickle

# import importlib
# importlib.reload(Energy.Models)

In [ ]:
df = get_dax_data()

In [ ]:
# We have to start with the tuesday data
start_date_excl = most_recent_thursday(df) - timedelta(days=1)
df_cval = df.loc[df.index < start_date_excl]
df_cval.index = df_cval.index.date

In [ ]:
df_cval.tail()

In [123]:
def evaluate_models(models, df, last_x, years =False, months=False, weeks=False):
    # Check that exactly one of the boolean parameters is True
    if sum([years, months, weeks]) != 1:
        raise ValueError("Exactly one of the boolean parameters (years, months, weeks) must be True.")
    
    years = int(years)
    months = int(months)
    weeks = int(weeks)
        
    for m in models:
        print(f'*********** Start the evaluation of Model {m["name"]} ***********')
        m['evaluation'] = evaluate_model(m, df, last_x, years, months, weeks)
        
def evaluate_model(model, df, last_x, years, months, weeks):
    df_before = df
    evaluation = pd.DataFrame()
    
    for w in range(last_x):
        print(f'Iteration {w+1} of {last_x}')
        df_before, df_after = split_time(df_before, num_years=years, num_months=months, num_weeks=weeks)
        pred = model['function'](df_before)
        # Makes sure we try to find observations for dates that have an observation (e.g. 05-01 is missing)
        dates = [d for d in pred['forecast_date'] if d in df.index]
        obs = pd.DataFrame({'ret1': df.loc[dates]['ret1']})
        pred = pred.set_index('forecast_date')
        obs.index = obs.index.strftime('%Y-%m-%d')
        # obs.index = obs.index.date
        # pred.index = pd.to_datetime(pred.index).date
        merged_df = pd.merge(pred, obs, left_index=True, right_index=True) 
    
         # Add scores to the merged_df
        for index, row in merged_df.iterrows():
            quantile_preds = row[['q0.025','q0.25','q0.5','q0.75','q0.975']]
            observation = row['ret1']
            score = evaluate_horizon(quantile_preds, observation)
            merged_df.at[index, 'score'] = score
        # print(merged_df[['q0.025','q0.25','q0.5','q0.75','q0.975']])
        evaluation = pd.concat([evaluation, merged_df])
    evaluation = evaluation.sort_index()
    return evaluation

In [124]:
models = [
    {
        'name': 'baseline',
        'function': baseline
     },
]


In [125]:
evaluate_models(models, df, 100, weeks=True)

*********** Start the evaluation of Model baseline ***********
Iteration 1 of 100
Iteration 2 of 100
Iteration 3 of 100
Iteration 4 of 100
Iteration 5 of 100
Iteration 6 of 100
Iteration 7 of 100
Iteration 8 of 100
Iteration 9 of 100
Iteration 10 of 100
Iteration 11 of 100
Iteration 12 of 100
Iteration 13 of 100
Iteration 14 of 100
Iteration 15 of 100
Iteration 16 of 100
Iteration 17 of 100
Iteration 18 of 100
Iteration 19 of 100
Iteration 20 of 100
Iteration 21 of 100
Iteration 22 of 100
Iteration 23 of 100
Iteration 24 of 100
Iteration 25 of 100
Iteration 26 of 100
Iteration 27 of 100
Iteration 28 of 100
Iteration 29 of 100
['2023-05-02', '2023-05-03', '2023-05-04', '2023-05-05']
0    2023-05-01
1    2023-05-02
2    2023-05-03
3    2023-05-04
4    2023-05-05
Name: forecast_date, dtype: object
Iteration 30 of 100
Iteration 31 of 100
Iteration 32 of 100
['2023-04-11', '2023-04-12', '2023-04-13']
0    2023-04-07
1    2023-04-10
2    2023-04-11
3    2023-04-12
4    2023-04-13
Name: forec

In [121]:
with open('./Model evaluations/baseline.pkl', 'wb') as f:
    pickle.dump(models, f)
#         
# with open('./Model evaluations/baseline.pkl', 'rb') as f:
#     models = pickle.load(f)